<a href="https://colab.research.google.com/github/dantepawn/Deep-Learning/blob/master/Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimators

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [0]:
fbdb = tf.keras.utils.get_file('I1.csv','http://www.football-data.co.uk/mmz4281/1819/I1.csv')

In [0]:
fb = pd.read_csv(fbdb)

In [10]:
fb['FTR'].value_counts()

H    166
D    108
A    106
Name: FTR, dtype: int64

In [0]:
train  = fb.drop(['FTR'] , axis = 1)
target = fb['FTR']

In [0]:
target.replace({'H':0,'D':1,'A':2}, inplace=True)

In [51]:
target

0      A
1      A
2      A
3      H
4      D
      ..
375    H
376    H
377    H
378    A
379    H
Name: FTR, Length: 380, dtype: object

In [18]:
train.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG',
       'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
       'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH',
       'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD',
       'VCA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA',
       'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh',
       'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA'],
      dtype='object')

In [19]:
train.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,I1,18/08/2018,Chievo,Juventus,2,3,1,1,D,7,23,2,11,7,9,0,8,2,0,0,0,13.00,5.75,1.25,14.00,6.25,1.22,13.00,5.80,1.25,15.87,6.21,1.25,13.00,6.00,1.22,15.00,6.00,1.25,41,15.87,13.70,6.44,5.88,1.27,1.24,38,1.75,1.70,2.24,2.13,19,2.00,1.68,1.64,2.38,2.29,18.84,6.42,1.22
1,I1,18/08/2018,Lazio,Napoli,1,2,1,1,D,9,11,5,6,8,5,4,7,0,0,0,0,2.80,3.40,2.50,2.80,3.50,2.45,2.80,3.55,2.40,2.90,3.59,2.48,2.80,3.50,2.40,2.88,3.40,2.50,41,2.95,2.82,3.65,3.50,2.56,2.44,38,1.74,1.68,2.29,2.17,20,0.00,2.12,2.07,1.83,1.79,2.78,3.57,2.59
2,I1,19/08/2018,Bologna,Spal,0,1,0,0,D,8,10,3,5,16,11,7,0,4,2,1,0,2.25,3.20,3.40,2.25,3.20,3.40,2.25,3.10,3.45,2.32,3.16,3.58,2.25,3.20,3.30,2.25,3.10,3.70,41,2.39,2.27,3.21,3.12,3.70,3.44,38,2.43,2.34,1.65,1.58,19,-0.25,1.97,1.92,1.99,1.94,2.31,3.18,3.59
3,I1,19/08/2018,Empoli,Cagliari,2,0,1,0,H,9,12,4,5,19,19,6,6,3,3,0,0,2.14,3.20,3.60,2.15,3.25,3.60,2.15,3.30,3.45,2.22,3.26,3.60,2.25,3.25,3.25,2.25,3.13,3.60,41,2.34,2.22,3.35,3.23,3.65,3.43,38,2.22,2.11,1.78,1.71,19,-0.25,1.98,1.91,1.98,1.94,2.54,3.42,2.95
4,I1,19/08/2018,Parma,Udinese,2,2,1,0,H,9,16,6,6,10,13,4,5,2,2,0,0,2.45,3.30,2.90,2.45,3.20,3.00,2.45,3.15,3.05,2.51,3.20,3.18,2.50,3.20,2.90,2.60,3.10,3.00,41,2.60,2.47,3.30,3.17,3.18,3.00,38,2.34,2.21,1.73,1.65,20,0.00,1.81,1.77,2.18,2.10,2.80,3.24,2.78


In [0]:
feats = [ 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA',  'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA']
feats.extend('B365H	B365D	B365A	BWH	BWD	BWA	IWH	IWD	IWA	PSH	PSD	PSA	WHH	WHD	WHA	VCH	VCD	VCA'.split())

In [0]:
train = train[feats]

In [45]:
train.head()

,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA
0,15.87,13.70,6.44,5.88,1.27,1.24,2.00,1.68,1.64,2.38,2.29,18.84,6.42,1.22,13.00,5.75,1.25,14.00,6.25,1.22,13.00,5.80,1.25,15.87,6.21,1.25,13.00,6.00,1.22,15.00,6.00,1.25
1,2.95,2.82,3.65,3.50,2.56,2.44,0.00,2.12,2.07,1.83,1.79,2.78,3.57,2.59,2.80,3.40,2.50,2.80,3.50,2.45,2.80,3.55,2.40,2.90,3.59,2.48,2.80,3.50,2.40,2.88,3.40,2.50
2,2.39,2.27,3.21,3.12,3.70,3.44,-0.25,1.97,1.92,1.99,1.94,2.31,3.18,3.59,2.25,3.20,3.40,2.25,3.20,3.40,2.25,3.10,3.45,2.32,3.16,3.58,2.25,3.20,3.30,2.25,3.10,3.70
3,2.34,2.22,3.35,3.23,3.65,3.43,-0.25,1.98,1.91,1.98,1.94,2.54,3.42,2.95,2.14,3.20,3.60,2.15,3.25,3.60,2.15,3.30,3.45,2.22,3.26,3.60,2.25,3.25,3.25,2.25,3.13,3.60
4,2.60,2.47,3.30,3.17,3.18,3.00,0.00,1.81,1.77,2.18,2.10,2.80,3.24,2.78,2.45,3.30,2.90,2.45,3.20,3.00,2.45,3.15,3.05,2.51,3.20,3.18,2.50,3.20,2.90,2.60,3.10,3.00


In [0]:
# function that creates a dataloader
def input_fn(features , labels , training = True , batch_size = 256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features) ,labels))
    if training :
     dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

In [0]:
my_feature_columns = [tf.feature_column.numeric_column(key=key) for key in train.keys()]
   

In [48]:
classifier = tf.estimator.DNNClassifier(hidden_units=[50,25],
                                          feature_columns= my_feature_columns,
                                          n_classes = 3                                    
                                          )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmprt2cxpr9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [53]:
classifier.train(
    input_fn= lambda: input_fn(train, target, training=True),
    steps=5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmprt2cxpr9/model.ckpt.
INFO:tensorflow:loss = 1.8307753, step = 0
INFO:tensorflow:global_step/sec: 86

In [56]:
classifier.evaluate( input_fn=lambda: input_fn(train , target , training=False) )

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-01-15T15:09:56Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmprt2cxpr9/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.34737s
INFO:tensorflow:Finished evaluation at 2020-01-15-15:09:57
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.55789477, averag

{'accuracy': 0.55789477,
 'average_loss': 0.9047844,
 'global_step': 5000,
 'loss': 0.9246965}

In [57]:
classifier.train(
    input_fn= lambda: input_fn(train, target, training=True),
    steps=5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmprt2cxpr9/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmprt2cxpr9/model.ckpt.
INFO:tensorflow:loss = 0.90583616, step = 5000
I

In [58]:
classifier.evaluate(lambda : input_fn(train , target , training = False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-01-15T15:26:28Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmprt2cxpr9/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.35022s
INFO:tensorflow:Finished evaluation at 2020-01-15-15:26:28
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.5736842, avera

{'accuracy': 0.5736842,
 'average_loss': 0.89654976,
 'global_step': 10000,
 'loss': 0.91657555}

# Linear Estimator